In [36]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [37]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/9章'
# os.chdir(path)

# ９章_小規模機械学習システムを作成する１０本ノック



### ノック８１：フォルダ生成をして初期の変数定義をしよう

In [39]:
import os
data_dir = "data"
input_dir = os.path.join(data_dir, "00_input")
store_monthly_dir = os.path.join(data_dir, "01_store_monthly")
ml_base_dir = os.path.join(data_dir, "02_ml_base")

output_ml_result_dir = os.path.join(data_dir, "10_output_ml_result")
output_report_dir = os.path.join(data_dir, "11_output_report")

master_dir = os.path.join(data_dir, "99_master")
model_dir = "models"

os.makedirs(input_dir, exist_ok=True)
os.makedirs(store_monthly_dir, exist_ok=True)
os.makedirs(ml_base_dir, exist_ok=True)
os.makedirs(output_ml_result_dir, exist_ok=True)
os.makedirs(output_report_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

##### ＊＊必ずデータやモデルの配置をおこなってください。＊＊

In [40]:
tg_ym = "202004"
target_file = "tbl_order_" + tg_ym + ".csv"
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
store_monthly_file = "store_monthly_data.csv"
ml_base_file = "ml_base_data.csv"

### ノック８２：更新データを読み込んで店舗別データを作成しよう

In [41]:
import pandas as pd
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
target_data = pd.read_csv(os.path.join(input_dir, target_file))

import datetime
max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


In [42]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 999]
    order_data = pd.merge(order_data, m_store, on='store_id', how='left')
    order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
    order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
    order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
    order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
    
    order_data.loc[:,'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:,'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:,'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    order_data.loc[:,'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:,'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'
    
    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[(order_data['status_name']=="お渡し済")|
                             (order_data['status_name']=="お支払済")].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[order_data['status_name']=="キャンセル"].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[order_data['takeout_name']=="デリバリー"].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[order_data['takeout_name']=="お持ち帰り"].groupby(['store_name']).count()[['order_id']]
    store_weekday = order_data.loc[order_data['weekday_info']=="平日"].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[order_data['weekday_info']=="休日"].groupby(['store_name']).count()[['order_id']]
    times = order_data['order_accept_hour'].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[order_data['order_accept_hour']==time].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis=1)
    store_delta = order_data.loc[order_data['status_name']!="キャンセル"].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']
    store_delta.columns = ['delta_avg']
    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_data = pd.concat([store_data, store_f, store_c, store_d, store_t, 
                        store_weekday, store_weekend, store_time, store_delta], axis=1)
    return store_data

In [43]:
store_data = data_processing(target_data)
store_data.reset_index(drop=False, inplace=True)
store_data.loc[:, "year_month"] = tg_ym
store_data.head(1)

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1147,945,202,848,299,842,305,92,125,...,101,97,105,103,97,110,109,96,34.520635,202004


### ノック８３：月次店舗データの更新をしよう

In [44]:
store_monthly_data = pd.read_csv(os.path.join(store_monthly_dir, store_monthly_file))
print(f'更新前：{len(store_monthly_data)}件')
store_monthly_data = pd.concat([store_monthly_data, store_data], ignore_index=True)
store_monthly_data.loc[:, "year_month"] = store_monthly_data["year_month"].astype(str)
store_monthly_data.drop_duplicates(subset=["store_name", "year_month"], inplace=True, keep="last")
print(f'更新後：{len(store_monthly_data)}件')
store_monthly_data.to_csv(os.path.join(store_monthly_dir, store_monthly_file), index=False)

更新前：2535件
更新後：2535件


### ノック８４：機械学習用データの作成と更新をしよう

In [45]:
from dateutil.relativedelta import relativedelta
y = store_monthly_data[["store_name", "year_month", "order_weekday", "order_weekend"]].copy()
y.loc[:, "one_month_ago"] = pd.to_datetime(y["year_month"], format="%Y%m")
y.loc[:, "one_month_ago"] = y["one_month_ago"].map(lambda x: x - relativedelta(months=1))
y.loc[:, "one_month_ago"] = y["one_month_ago"].dt.strftime("%Y%m")

y_one_month_ago = y.copy()
y_one_month_ago.rename(columns={"order_weekday":"order_weekday_one_month_ago", "order_weekend":"order_weekend_one_month_ago", "year_month":"year_month_for_join"}, inplace=True)

y = pd.merge(y, y_one_month_ago[["store_name", "year_month_for_join", "order_weekday_one_month_ago", "order_weekend_one_month_ago"]], left_on=["store_name", "one_month_ago"], right_on=["store_name", "year_month_for_join"], how="left")

y.dropna(inplace=True)
y.loc[y["order_weekday"] - y["order_weekday_one_month_ago"] > 0, "y_weekday"] = 1
y.loc[y["order_weekday"] - y["order_weekday_one_month_ago"] <= 0, "y_weekday"] = 0
y.loc[y["order_weekend"] - y["order_weekend_one_month_ago"] > 0, "y_weekend"] = 1
y.loc[y["order_weekend"] - y["order_weekend_one_month_ago"] > 0, "y_weekend"] = 0

y.rename(columns={"year_month":"target_year_month"}, inplace=True)
y = y[["store_name", "target_year_month", "one_month_ago", "y_weekday", "y_weekend"]].copy()
ml_data = pd.merge(y, store_monthly_data, left_on=["store_name", "one_month_ago"], right_on=["store_name", "year_month"], how="left")

del ml_data["target_year_month"]
del ml_data["one_month_ago"]
ml_data.head(3)


,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0,NaN,1147,945,202,841,306,844,303,...,101,95,107,106,100,108,109,96,34.110053,201904
1,さいたま南店,1.0,0.0,1504,1217,287,1105,399,1104,400,...,143,142,137,130,113,140,132,155,35.337716,201904
2,さいたま緑店,1.0,0.0,1028,847,181,756,272,756,272,...,95,102,82,90,93,95,95,84,34.291617,201904


In [46]:
ml_base_data = pd.read_csv(os.path.join(ml_base_dir, ml_base_file))
print(f'更新前：{len(ml_base_data)}件')
ml_base_data = pd.concat([ml_base_data, ml_data], ignore_index=True)
ml_base_data.loc[:, "year_month"] = ml_base_data["year_month"].astype(str)
ml_base_data.drop_duplicates(subset=["store_name", "year_month"], inplace=True, keep="last")
print(f'更新後：{len(ml_base_data)}件')
ml_base_data.to_csv(os.path.join(ml_base_dir, ml_base_file), index=False)

更新前：2145件
更新後：2340件


### ノック８５：機械学習モデル用の事前データ加工をしよう

### ノック８６：機械学習モデルの構築・評価をしよう

### ノック８７：新規データ予測に向けた下準備をしよう

### ノック８８：新規データの予測をしよう

### ノック８９：現場向けレポートを作成し出力しよう

#### ＊＊次ノックに行く前に、ノック81のtg_ymを、202005、202006、202007、202008の順番に指定し、実行してください。＊＊

### ノック９０：機械学習モデルの精度推移を可視化しよう